In [ ]:
"오늘 부산 날씨 어때?"  == "오늘 부산 날씨 알려줘"
"부산 날씨 오늘 어때?"  == "오늘 부산 날씨 알려줘"

In [ ]:
import pandas as pd
class2 = pd.read_csv("class2.csv")

print(class2)

from sklearn import preprocessing 
label_encoder = preprocessing.LabelEncoder()
onehot_encoder = preprocessing.OneHotEncoder()

train_x = label_encoder.fit_transform(class2['class2'])
train_x

   Unnamed: 0      id tissue class class2      x      y      r
0           0  mdb000      C  CIRC      N  535.0  475.0  192.0
1           1  mdb001      A  CIRA      N  433.0  268.0   58.0
2           2  mdb002      A  CIRA      I    NaN    NaN    NaN
3           3  mdb003      C  CIRC      B    NaN    NaN    NaN
4           4  mdb004      F  CIRF      I  488.0  145.0   29.0
5           5  mdb005      F  CIRF      B  544.0  178.0   26.0


array([2, 2, 1, 0, 1, 0])

In [ ]:
import numpy as np
# np.eye(3)[[0,0,0]]
onehot = np.eye(3)[train_x]
print(onehot)

[[0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]


In [ ]:
"i love food"    #   I->[1,0,0]            love->[0,1,0]      food->[0.0.1]

In [ ]:
# 벡터의 내적   v1= (1,2),  v2(3,4)    1*3+2*4 = 3+ 8 = 11
#  (1,0,0) (0,1,0) = 0

# 횟수 기반 임베딩

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = [
    'This is last chance. My name is alice',
    'and if you do not have this chance.',
    'you will never get any chance.',
    'will you do get this one?',
    'please, get this chance',
]

vect = CountVectorizer()
vect.fit(corpus)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [ ]:
# 어휘집 확인
vect.vocabulary_

{'alice': 0,
 'and': 1,
 'any': 2,
 'chance': 3,
 'do': 4,
 'get': 5,
 'have': 6,
 'if': 7,
 'is': 8,
 'last': 9,
 'my': 10,
 'name': 11,
 'never': 12,
 'not': 13,
 'one': 14,
 'please': 15,
 'this': 16,
 'will': 17,
 'you': 18}

In [ ]:
# 문장 임베딩
a = vect.transform(['you will never get any chance.',
                    'never get any chance.',
                    'my name is alice'
                    ]).toarray()
print(a.shape)
print(a)

(3, 15)
[[0 1 1 0 1 0 0 0 0 0 1 0 0 1 1]
 [0 1 1 0 1 0 0 0 0 0 1 0 0 0 0]
 [1 0 0 0 0 0 0 0 1 1 0 0 0 0 0]]


In [ ]:
# 불용어 처리
vect = CountVectorizer(stop_words=["and", "is", "please", "this"]).fit(corpus)
vect.vocabulary_

{'alice': 0,
 'any': 1,
 'chance': 2,
 'do': 3,
 'get': 4,
 'have': 5,
 'if': 6,
 'last': 7,
 'my': 8,
 'name': 9,
 'never': 10,
 'not': 11,
 'one': 12,
 'will': 13,
 'you': 14}

# cosine 유사도

In [ ]:
v1 = np.array([1,1])
v2 = np.array([7,7])

print(np.sum(np.abs(v1 - v2)  )) # city block distance
print(np.sqrt(np.sum((v1 - v2)**2)))  #유클리디안 )
print(   np.sum( v1*v2 )    )
print(   np.sqrt(np.sum( v1**2 )) *  np.sqrt(np.sum( v2**2 ))    )
print(  np.sum( v1*v2 )  / (np.sqrt(np.sum( v1**2 )) *  np.sqrt(np.sum( v2**2 )) )   )  # 코사인유사도




12
8.48528137423857
14
14.000000000000002
0.9999999999999999


In [ ]:
from scipy.spatial import distance

1 - distance.cosine(v1, v2)

1.0

In [ ]:
print(1 - distance.cosine(a[0], a[0]))  # 동일한 문장
print(1 - distance.cosine(a[0], a[1]))  # 81% 정도 유사
print(1 - distance.cosine(a[0], a[2]))  # 완전 다름

1.0
0.8164965809277259
0.0


In [ ]:
유사도 : 0이면 서로다름      1:동일
거리   : 0이면 서도동일      1:서로다름

유사도 = 1 - 거리  
거리 = 1 - 유사도

# TF-IDF
- TF : 문장(doc)에서 단어의 빈도수
- DF : 단어가 나타난 문장(문서)의 수
- IDF : 1/DF
- TF * IDF

- 효과 : 서로 다른 문장들이 잘 구분될 수 있도록 함

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [ ]:
sent = ["오늘 휴일", 
        "휴일 오늘", 
        "휴일 인 오늘 도 서쪽 을 중심 으로 폭염 이 이어졌는데요, 내일 은 반가운 비 소식 이 있습니다.", 
        "폭염 을 피해서 휴일 에 놀러왔다가 갑작스런 비 로 인해 망연자실 하고 있습니 다.", 
        " 내일 은 반가운 비 소식 이 있습니다."] 

tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(sent)


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

3.6.0


In [ ]:
tfidf = tfidf_vectorizer.transform(sent)
print(tfidf.toarray())

[[0.         0.         0.         0.         0.         0.
  0.         0.76524053 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.64374446]
 [0.         0.         0.         0.         0.         0.
  0.         0.76524053 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.64374446]
 [0.         0.28487999 0.         0.         0.28487999 0.3531014
  0.28487999 0.23647612 0.3531014  0.3531014  0.         0.
  0.28487999 0.3531014  0.28487999 0.         0.         0.19893117]
 [0.3542556  0.         0.3542556  0.3542556  0.         0.
  0.         0.         0.         0.         0.3542556  0.3542556
  0.         0.         0.28581119 0.3542556  0.3542556  0.19958143]
 [0.         0.5        0.         0.         0.5        0.
  0.5        0.         0.         0.         0.         0.
  0.5        0.         0.         0.         0.         0.        ]]


In [ ]:
tfidf_vectorizer.transform(['이상한 휴일']).toarray()

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1.]])

# 문서 유사도 비교

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

#cosine_similarity(tfidf[0:1], tfidf[0:1])
#cosine_similarity(tfidf[0:1], tfidf)
cosine_similarity(tfidf, tfidf)   #   5x5 행렬    문장수x문장수

array([[1.        , 1.        , 0.30902195, 0.12847944, 0.        ],
       [1.        , 1.        , 0.30902195, 0.12847944, 0.        ],
       [0.30902195, 0.30902195, 1.        , 0.12112485, 0.56975997],
       [0.12847944, 0.12847944, 0.12112485, 1.        , 0.        ],
       [0.        , 0.        , 0.56975997, 0.        , 1.        ]])

# Q1)문서 분류
- 스포츠 관련 문서(100건)
- 예술 관련 문서(100건)
- 정치 관련 문서(100건)


- 300개 데이터, 1000개단어
- X : 300 by 1000
- y = [0,0, .....    1,1,       2,2  ...]  # 300,
- X,y 이용해서 학습한다.

- 임의문서,  transform -> 1x1000 행렬
- model.predict()

In [ ]:
sent = ["""

아이뉴스24 류한준 기자] 이제부터는 단판 승부다. 스테파노 라바리니 감독이 지휘봉을 잡고 있는 한국 여자배구대표팀은 오는 4일 도쿄올림픽 여자배구 8강전을 치른다.
터키를 만난다. 8강에 오른 팀 중에서 어느 하나 만만한 상대는 없다. 한국은 터키와 상대 전적에서 2승 7패로 밀리고 있다. 도쿄올림픽에 앞서 열린 국제배구연맹(FIVB) 주최 2021 발리볼 네이션스리그(VNL)에서도 패했다.
그러나 낯선 상대는 아니다. 라바리니 감독은 클럽팀을 이끌고 터키리그 소속팀들과 유럽배구연맹(CEV) 주최 컵대회와 챔피언스리그 등에서 경기를 치른 경험이 있다.
""",
"""
윤석열이 정치에 입문한다면 가장 중요한, 절대 놓치지 말아야 할 것은 "반문"의 상징과도 같은 이미지입니다. 윤석열은 조국 사태 수사하고, 추미애한테 탄압받고, 문재인 정권과 맞서 싸우면서 반문 진영의 상징, 혹은 반문 투사 같은 이미지를 갖게 되어버렸습니다. 이게 윤석열의 가장 큰 정치적 자산입니다. 어설프게 문재인과 타협하려 하는 순간, 윤석열은 정치인으로서의 가치를 잃게 됩니다. 다행히 언론과 측근들을 통해 나오는 메세지를 보니 윤석열이 그걸 잘 알고 있는거 같긴 합니다. 
"""
] 

# Q2) Q&A 시스템

Q                                    A
전화번가 머죠                        전화번호는 xxx입니다.
전화전호 알려주세요                  전화번호는 xxx입니다.

- 학습데이터 이용해 학습(Q)


사용자 : 질문, transform
전체 Q와 유사도 계산, 가장 유사한 질문에 대한 답을 리턴


# Q3) 내용 기반 영화 추천 시스템
- 줄거리를 기반으로 tfidf으로 줄거리를 모델링하고
- 내가 좋아하는 영화의 줄거리와 유사도 비교해 추천

In [ ]:
corpus = [
    'you know I want your love',      
    'I like you',
    'what should I do ',    
]
vector = CountVectorizer()


str = '그러나까 오늘 부산 날씨가 모야' 

# 불용어 처리
str = getWord(str, stopwords) 

# 형태소 분석 & 문자열로 추출

sentence =  [word[0] for  word in twitter.pos(str, norm=True, stem=True)  ]
str = " ".join(sentence)

# word2vec

In [ ]:
import gensim
from gensim.models import Word2Vec

sentences = [
                ['this', 'is', 'a',   'good',      'product'],
                ['it',   'is', 'a',   'excellent', 'product'],
                ['it',   'is', 'a',   'bad',       'product'],
                ['that', 'is', 'the', 'worst',     'product']
            ]

model = Word2Vec(sentences, size = 20, window=3, min_count=1)  # 4.0버전 vector_size=2

In [ ]:
word_vectors = model.wv
print(word_vectors.vocab.keys())
vocabs = word_vectors.vocab.keys()
# vocabs =  model.wv.index_to_key  # 4.0버전

dict_keys(['this', 'is', 'a', 'good', 'product', 'it', 'excellent', 'bad', 'that', 'the', 'worst'])


In [ ]:
word_vectors['this']

array([ 0.12349486, -0.11331619], dtype=float32)

In [ ]:
word_vectors_list = [word_vectors[v] for v in vocabs]
print(word_vectors_list)

[array([ 0.12349486, -0.11331619], dtype=float32), array([-0.03542474,  0.04836241], dtype=float32), array([-0.23788716, -0.16577792], dtype=float32), array([ 0.21124907, -0.16378489], dtype=float32), array([ 0.15977931, -0.2333205 ], dtype=float32), array([-0.0408023,  0.1708948], dtype=float32), array([-0.24199414,  0.22828929], dtype=float32), array([ 0.13528685, -0.03029034], dtype=float32), array([ 0.1345684 , -0.11077366], dtype=float32), array([-0.24184752, -0.2098983 ], dtype=float32), array([ 0.24197088, -0.11807898], dtype=float32)]


# 유사도

In [ ]:
model.wv.similarity('bad', 'good')

-0.021200009

In [ ]:
model.wv.most_similar('it', topn=10)

[('excellent', 0.3024637699127197),
 ('a', 0.09022259712219238),
 ('that', 0.06555291265249252),
 ('is', 0.042932942509651184),
 ('this', 0.027582742273807526),
 ('the', -0.03926095366477966),
 ('good', -0.07508274912834167),
 ('worst', -0.08592996001243591),
 ('bad', -0.19587333500385284),
 ('product', -0.5923073887825012)]

# 모델 저장/로딩

In [ ]:
model.save('model.w2v')

model2 = Word2Vec.load('model.w2v')

model2.wv['it']

array([-0.00408023,  0.01708949, -0.01864988,  0.02303149, -0.02089864,
        0.00835645, -0.02082762,  0.02031628, -0.02362515, -0.02163178,
        0.00577815, -0.01201425, -0.01999409,  0.01761933,  0.00256586,
        0.01119068,  0.02474074,  0.01153428, -0.01777749, -0.00405814],
      dtype=float32)

In [ ]:
ko_model = Word2Vec.load('ko.bin')

In [ ]:
ko_model.wv.vocab.keys()

dict_keys(['관위', '정어리', '유식론', '장로회', '춘추관', '도입부', '민병', '어렵', '매니저', '청담', '광주시', '고민', '어로', '어록', '폴라리스', '십만', '관음', '효과적', '관의', '당뇨병', '영중추', '듣', '민법', '마두라', '중국계', '여덟째', 'ㄴ단', 'ㄴ다', '테스트', '어찌하', '정안국', '엎드리', '싱가포르', '벡터', '관자', '관작', '팽대부', '테이블', '분석학', '유장', '렙톤', '어뢰', '다다르', '제공자', '히트', '멸망', '암기', '마르텔', '뻔', '윈도', '민방', '훤', '분기점', '결탁', '완결', '생매장', '워털루', '청도', '관영', '원자핵', '놋', '국정원장', '청동', '표면적', '저전력', '일치', '몽', '천동설', '어류', '마르타', '깊', '잘생기', '중재자', '관왕', '보퍼트', '현내면', '유임', '어른', '아와지', '머지', '수용자', '여진족', '마르탱', '피오르드', '사법권', '유재', '잠수함', '돌연', '성화', '펌', '여쭈', '처치', '간균', '섬광', '일컫', 'ㄴ대', '어림', '안토니', '관용', '어리', '암꽃', '청대', '앨런', '말', '관원', '게라', '다음달', '전염성', '마르크', 'ㄴ데', '영부인', '아마미', '소피스트', '스탠드', '폐회', '폐해', '베누스', '폐합', '설립자', '럭키', '다다미', '폐하', '서해안', '식초', '스크린', '잉크', '여주', '파나마', '이화학', '여종', '관아', '관악', '맺었으', '멸문', '지침서', '싫', '휘말리', '무선', '송정동', '은은', '수준의', '잉태', '로시니', '윙', '은의', '어명', '면상', '특허권', '민비', '섬기', '비상시', '비상식', '폐허', '원시림', 

In [ ]:
ko_model.wv.most_similar("강아지")

[('고양이', 0.7290452718734741),
 ('거위', 0.7185635566711426),
 ('토끼', 0.7056223154067993),
 ('멧돼지', 0.6950401067733765),
 ('엄마', 0.6934334635734558),
 ('난쟁이', 0.6806551218032837),
 ('한마리', 0.6770296096801758),
 ('아가씨', 0.6750352382659912),
 ('아빠', 0.6729634404182434),
 ('목걸이', 0.6512460708618164)]

In [ ]:
 ko_model.wv.most_similar("컴퓨터")

[('하드웨어', 0.7727354764938354),
 ('소프트웨어', 0.7308895587921143),
 ('마이크로프로세서', 0.7205492258071899),
 ('개인용', 0.7154535055160522),
 ('노트북', 0.7130324244499207),
 ('계산기', 0.7044994831085205),
 ('하드디스크', 0.6943572759628296),
 ('태블릿', 0.6915029287338257),
 ('그래픽스', 0.6882646083831787),
 ('프린터', 0.6876466274261475)]

In [ ]:
!pwd  #  /content

/content


In [ ]:
ko_model = Word2Vec.load('/content/drive/MyDrive/data/ko.bin')

In [ ]:
from google.colab import drive
drive.mount('/content/mnt')

Mounted at /content/mnt
